# Sources:
- https://www.youtube.com/watch?v=K21BSZPFIjQ

Extract selected mails from your gmail account

1. Make sure you enable IMAP in your gmail settings
(Log on to your Gmail account and go to Settings, See All Settings, and select
 Forwarding and POP/IMAP tab. In the "IMAP access" section, select Enable IMAP.)

2. If you have 2-factor authentication, gmail requires you to create an application
specific password that you need to use. 
Go to your Google account settings and click on 'Security'.
Scroll down to App Passwords under 2 step verification.
Select Mail under Select App. and Other under Select Device. (Give a name, e.g., python)
The system gives you a password that you need to use to authenticate from python.

In [1]:
# Importing libraries
import re
import imaplib
import email
import pandas as pd
import yaml  # To load saved login credentials from a yaml file

In [2]:
with open("credentials.yml") as f:
    content = f.read()

# from credentials.yml import user name and password
my_credentials = yaml.load(content, Loader=yaml.FullLoader)

# Load the user name and passwd from yaml file
user, password = my_credentials["user"], my_credentials["password"]

# URL for IMAP connection
imap_url = 'imap.gmail.com'

# Connection with GMAIL using SSL
my_mail = imaplib.IMAP4_SSL(imap_url)

# Log in using your credentials
my_mail.login(user, password)

# Select the Inbox to fetch messages
my_mail.select('Inbox')

# Define Key and Value for email search
# For other keys (criteria): https://gist.github.com/martinrusev/6121028#file-imap-search
key = 'FROM'
value = 'dan@tldrnewsletter.com'
# Search for emails with specific key and value
_, data = my_mail.search(None, key, value)

mail_id_list = data[0].split()  # IDs of all emails that we want to fetch

msgs = []  # empty list to capture all messages
# Iterate through messages and extract data into the msgs list
for num in mail_id_list:
    # RFC822 returns whole message (BODY fetches just body)
    typ, data = my_mail.fetch(num, '(RFC822)')
    msgs.append(data)

# Now we have all messages, but with a lot of details
# Let us extract the right text and print on the screen

# In a multipart e-mail, email.message.Message.get_payload() returns a
# list with one item for each part. The easiest way is to walk the message
# and get the payload on each part:
# https://stackoverflow.com/questions/1463074/how-can-i-get-an-email-messages-text-content-using-python

# NOTE that a Message object consists of headers and payloads.
dic = {}
for msg in msgs[::-1]:
    for response_part in msg:
        if type(response_part) is tuple:
            my_msg = email.message_from_bytes((response_part[1]))
            for part in my_msg.walk():
                # print(part.get_content_type())
                if part.get_content_type() == 'text/plain':
                    payload = part.get_payload(decode=True).decode(part.get_content_charset('utf-8'))
                    dic[my_msg.get("Date")] = re.findall(r'https?://.*', payload)


df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in dic.items()]))

In [3]:
df

,"Tue, 20 Feb 2024 12:20:16 +0000","Tue, 20 Feb 2024 12:14:29 +0000","Tue, 20 Feb 2024 12:07:46 +0000","Tue, 20 Feb 2024 11:34:00 +0000","Tue, 20 Feb 2024 11:10:02 +0000","Mon, 19 Feb 2024 14:21:56 +0000","Mon, 19 Feb 2024 14:09:49 +0000","Mon, 19 Feb 2024 13:15:15 +0000","Mon, 19 Feb 2024 13:11:35 +0000","Mon, 19 Feb 2024 13:10:48 +0000","Mon, 19 Feb 2024 12:13:14 +0000","Mon, 19 Feb 2024 12:12:45 +0000","Mon, 19 Feb 2024 11:31:24 +0000"
0,https://tldr.tech/webdev?ref=9d5a9676-ccf7-11e...,https://tldr.tech/devops?ref=9d27e85c-ccf7-11e...,https://tldr.tech/marketing?ref=9d57216c-ccf7-...,https://tldr.tech/tech?ref=6261134 [19]\r,https://tldr.tech/product?ref=9d601132-ccf7-11...,https://tldr.tech/ai?ref=9d56edaa-ccf7-11ee-9a...,https://tldr.tech/infosec?ref=9d5e4ae6-ccf7-11...,https://tldr.tech/crypto?ref=9d2a457a-ccf7-11e...,https://tldr.tech/founders?ref=9d55fecc-ccf7-1...,https://tldr.tech/design?ref=9d27d326-ccf7-11e...,https://tldr.tech/webdev?ref=9d5a9676-ccf7-11e...,https://tldr.tech/marketing?ref=9d57216c-ccf7-...,https://tldr.tech/tech?ref=6261134 [19]\r
1,https://tldr.tech/webdev?utm_source=tldrwebdev...,https://tldr.tech/devops?utm_source=tldrdevops...,https://tldr.tech/marketing?utm_source=tldrmar...,https://tldr.tech/signup?utm_source=tldr&ref=6...,https://tldr.tech/product?utm_source=tldrprodu...,https://tldr.tech/ai?utm_source=tldrai&ref=9d5...,https://tldr.tech/infosec?utm_source=tldrinfos...,https://tldr.tech/crypto?utm_source=tldrcrypto...,https://tldr.tech/founders?utm_source=tldrfoun...,https://tldr.tech/design?utm_source=tldrdesign...,https://tldr.tech/webdev?utm_source=tldrwebdev...,https://tldr.tech/marketing?utm_source=tldrmar...,https://tldr.tech/signup?utm_source=tldr&ref=6...
2,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...,https://share.hsforms.com/1hmOFVmqOTrON_SRvaRq...
3,https://advertise.tldr.tech/?utm_source=tldrwe...,https://advertise.tldr.tech/?utm_source=tldrde...,https://advertise.tldr.tech/?utm_source=tldrma...,https://advertise.tldr.tech/?utm_source=tldr&u...,https://advertise.tldr.tech/?utm_source=tldrpr...,https://advertise.tldr.tech/?utm_source=tldrai...,https://advertise.tldr.tech/?utm_source=tldrin...,https://advertise.tldr.tech/?utm_source=tldrcr...,https://advertise.tldr.tech/?utm_source=tldrfo...,https://advertise.tldr.tech/?utm_source=tldrde...,https://advertise.tldr.tech/?utm_source=tldrwe...,https://advertise.tldr.tech/?utm_source=tldrma...,https://advertise.tldr.tech/?utm_source=tldr&u...
4,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...,https://actions.tldrnewsletter.com/web-version...
5,https://www.postman.com/postcon/?utm_source=tl...,https://www.postman.com/postcon/?utm_source=tl...,https://www.androidpolice.com/reddit-deal-unna...,https://www.thewrap.com/chatgpt-growth-2024/?u...,https://www.moderntreasury.com/resources/how-t...,https://landing.deepset.ai/gartner-how-to-pilo...,https://sec